In [1]:
import numpy as np
import matplotlib.pyplot as plt
import SpectralSynthesis as ss
import pickle
from tqdm import tqdm

In [2]:
N_X = 22000
H_RANGE = [0,1]
S_RANGE = [0,3]
N_PIX = 128
E_DIMENSIONS = 2

PERIODIC = False
NOISE = True
EXP = True

In [3]:
H_list = np.random.uniform(H_RANGE[0], H_RANGE[1], N_X)
S_list = np.random.uniform(S_RANGE[0], S_RANGE[1], N_X)
noise_pct_list = np.random.random(N_X)*0.01

In [ ]:
parameters = np.array([H_list, S_list, noise_pct_list]).T
parameters[0]

array([0.23678909, 0.82230982, 0.00096583])

In [ ]:
def generate_field(H,S,NOISE_FRACTION, N_PIX, PERIODIC, EXP):
    if PERIODIC:
        field = ss.fBm(E=E_DIMENSIONS,exp=EXP,H=H,sigma=S,N=N_PIX, centred = False)
    else:
        field = ss.fBm(E=E_DIMENSIONS,exp=EXP,H=H,sigma = S, N=N_PIX*4, centred=False)
        field = field[int((N_PIX*4-N_PIX)/2):int((N_PIX*4+N_PIX)/2),int((N_PIX*4-N_PIX)/2):int((N_PIX*4+N_PIX)/2)]

    m_1_field = np.mean(field)
    s_1_field = np.std(field)
    m_2_field = 0
    s_2_field = 1/4

    field *= s_2_field/s_1_field
    field += (m_2_field-m_1_field*s_2_field/s_1_field)

    if NOISE:
        noise = ss.fBm(E=E_DIMENSIONS,exp=False,H=-1,N=N_PIX)

        m_1_noise = np.mean(noise)
        s_1_noise = np.std(noise)
        m_2_noise = 0
        s_2_noise = s_2_field*NOISE_FRACTION

        noise *= s_2_noise/s_1_noise
        noise += (m_2_noise-m_1_noise*s_2_noise/s_1_noise)

        field += noise

    m_1_field = np.mean(field)
    s_1_field = np.std(field)
    m_2_field = 0
    s_2_field = 1/4

    field *= s_2_field/s_1_field
    field += (m_2_field-m_1_field*s_2_field/s_1_field)

    field[np.where(field>1.)] = 1.
    field[np.where(field<-1.)] = -1.

    field = (field-np.min(field))/(np.max(field)-np.min(field))

    return field

DATA = [(generate_field(*parameter, N_PIX, PERIODIC, EXP), parameter) for parameter in tqdm(parameters)]

DATA

  0%|                                                                                        | 0/22000 [00:00<?, ?it/s]C:\Users\C1764397\Workshop\PhD\Scripts\SpectralSynthesis.py:58: RuntimeWarning: divide by zero encountered in power
  rad = np.where(k>0.0,k**(-(beta*0.5)),0.0)
  7%|█████                                                                        | 1453/22000 [02:52<42:22,  8.08it/s]

In [ ]:
dbfile = open('D:\FBM Training Data\X.pickle', 'ab') 

pickle.dump(DATA, dbfile)                      
dbfile.close() 